In [2]:
from pynq import Overlay
import pynq.lib.dma
import numpy as np
from pynq import Xlnk

ModuleNotFoundError: No module named 'pynq'

In [2]:
# bias = convolutional bias
# weight_3d = 3 x 3 x 3 sized list containing the weight
#             values
# image_3d = 3 dimensional list representing the image,
#            size 3 x 32 x 32 with zero-padding
def FeedForwardConv0(bias, weight_3d, image_3d):    
    pad = []
    
    # Add the padding to the image
    for i in range(3):
        pad.append(np.zeros((34, 34), dtype=np.int32))
        pad[-1][1:33, 1:33] = np.asarray(image_3d[i], dtype=np.int32)
        pad[-1] = pad[-1].flatten().tolist()

    # Flatten the weight so we can feed it through serially
    # into the hardware
    flat_weight = []
    for i in range(3):
        flat_weight.append([weight for weight_2d in weight_3d[i] for weight in weight_2d])
    
    # 'samples' is what we will send through the hardware
    feed = [bias]
    feed = feed + flat_weight[0] + pad[0]
    feed = feed + flat_weight[1] + pad[1]
    feed = feed + flat_weight[2] + pad[2]
    feed = feed + [0]
    samples = np.asarray(feed).astype(np.int32)
    slen = len(samples)
    
    # Send the data through
    overlay = Overlay('fp.bit')
    dma = overlay.axi_dma_0
    overlay.reset()
    
    xlnk = Xlnk()
    out_buffer = xlnk.cma_array(shape=(slen+5+1+1,), dtype=np.int32)
    in_buffer = xlnk.cma_array(shape=(slen,), dtype=np.int32)
    np.copyto(in_buffer, samples)
    
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    
    # Cut out the useless values returned from the hardware
    # the pipelining creates some "unintuitive" slicing
    out = []
    for i in range(32):
        off_index = 2330 + (i * 34) + 66 + 5 + 9 + 5 + 1 
        out = out + list(out_buffer[off_index : off_index + 32])

    # return the convolutional result
    out = np.asarray(out).reshape((32, 32))
    return out

#### Test the hardware

In [3]:
# Test the hardware using a weight of ones and an image of 
# a reshaped range(32, 32) with zero padding

bias = 3
weight = np.ones((3, 3, 3)).tolist()
image = np.asarray(list(range(32*32))*3, dtype=np.int32).reshape((3, 32,32)).tolist()

print('Hardware Output: ')
print(FeedForwardConv0(bias, weight, image))

[[  201   309   327 ...,   813   831   561]
 [  588   894   921 ...,  1650  1677  1128]
 [ 1164  1758  1785 ...,  2514  2541  1704]
 ..., 
 [16716 25086 25113 ..., 25842 25869 17256]
 [17292 25950 25977 ..., 26706 26733 17832]
 [11721 17589 17607 ..., 18093 18111 12081]]


#### Generate the correct output for comparison

In [8]:
import numpy as np

def convPad(filt, image):
    out = np.zeros((32, 32))
    for i in range(34 - 2):
        for j in range(34 - 2):
            s = 0
            s += filt[0][0] * image[i][j]
            s += filt[0][1] * image[i][j + 1]
            s += filt[0][2] * image[i][j + 2]
            s += filt[1][0] * image[i + 1][j]
            s += filt[1][1] * image[i + 1][j + 1]
            s += filt[1][2] * image[i + 1][j + 2]
            s += filt[2][0] * image[i + 2][j]
            s += filt[2][1] * image[i + 2][j + 1]
            s += filt[2][2] * image[i + 2][j + 2]
            out[i][j] = s
    return out

filt = np.ones((3, 3))
i = np.asarray(range(32*32), dtype=np.int32).reshape((32, 32))
j = np.zeros((34, 34))
j[1:33, 1:33] = i

print('correct output: ')
out = convPad(filt, j)*3 + 3
print(out)

[[  201.   309.   327. ...   813.   831.   561.]
 [  588.   894.   921. ...  1650.  1677.  1128.]
 [ 1164.  1758.  1785. ...  2514.  2541.  1704.]
 ...
 [16716. 25086. 25113. ... 25842. 25869. 17256.]
 [17292. 25950. 25977. ... 26706. 26733. 17832.]
 [11721. 17589. 17607. ... 18093. 18111. 12081.]]
